# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  Jul  8 2022 10:22AM  241389        20  SSNA-8248028   
1  Jul  8 2022 10:22AM  241387        10       8244168   
2  Jul  8 2022 10:22AM  241387        10       8244169   
3  Jul  8 2022 10:22AM  241387        10       8244104   
4  Jul  8 2022 10:22AM  241387        10       8244924   
5  Jul  8 2022 10:22AM  241387        10       8245467   
6  Jul  8 2022 10:22AM  241387        10       8245468   
7  Jul  8 2022 10:18AM  241388        21     619980-01   
8  Jul  8 2022 10:05AM  241386        10    Eme-100829   
9  Jul  8 2022  9:52AM  241378        15    PNC363338B   

                           Customer ShipmentStatus  
0  Sartorius Stedim North America         Released  
1                  Eywa Pharma Inc.       Released  
2                  Eywa Pharma Inc.       Released  
3                  Eywa Pharma Inc.       Released  
4                  Eywa Pharma Inc.       Released  
5                  Eywa Pharma Inc.       Released  
6                  Eywa Pharma Inc.       Released  
7                 NBTY Global, Inc.       Released  
8                       Emerginnova       Released  
9              Person & Covey, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
42  241383       Released          1
43  241386       Released          1
44  241387       Released          6
45  241388       Released          1
46  241389       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
241383                NaN        6.0       1.0
241386                NaN        NaN       1.0
241387                NaN        NaN       6.0
241388                NaN        NaN       1.0
241389                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241297                0.0        0.0       1.0
241298                0.0        0.0       1.0
241306                0.0        0.0       1.0
241319               25.0       15.0       3.0
241320                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241297                  0          0         1
241298                  0          0         1
241306                  0          0         1
241319                 25         15         3
241320                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               241297          0          0         1
1               241298          0          0         1
2               241306          0          0         1
3               241319         25         15         3
4               241320          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               241297                            1
1               241298                            1
2               241306                            1
3               241319        25        15        3
4               241320                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                          Customer
0    Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America  
1    Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.
7    Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.
8    Jul  8 2022 10:05AM  241386        10                       Emerginnova
9    Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.
177  Jul  8 2022  9:34AM  241381        10                 ISDIN Corporation
216  Jul  8 2022  9:31AM  241383        15             Alliance Pharma, Inc.
223  Jul  8 2022  9:29AM  241379        15            Mizner Bioscience, LLC
234  Jul  8 2022  9:29AM  241382        10                 ISDIN Corporation
242  Jul  8 2022  9:26AM  241380        12                         Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                          Customer  \
0  Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America     
1  Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.   
2  Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.   
3  Jul  8 2022 10:05AM  241386        10                       Emerginnova   
4  Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.   
5  Jul  8 2022  9:34AM  241381        10                 ISDIN Corporation   
6  Jul  8 2022  9:31AM  241383        15             Alliance Pharma, Inc.   
7  Jul  8 2022  9:29AM  241379        15            Mizner Bioscience, LLC   
8  Jul  8 2022  9:29AM  241382        10                 ISDIN Corporation   
9  Jul  8 2022  9:26AM  241380        12                         Hush Hush   

  Completed Executing Released  
0                            1  
1                            6  
2                            1  
3                            1  
4                   9      159  
5                           39  
6                   6        1  
7                   2        9  
8                            8  
9                           10

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                          Customer  \
0  Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America     
1  Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.   
2  Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.   
3  Jul  8 2022 10:05AM  241386        10                       Emerginnova   
4  Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.   
5  Jul  8 2022  9:34AM  241381        10                 ISDIN Corporation   
6  Jul  8 2022  9:31AM  241383        15             Alliance Pharma, Inc.   
7  Jul  8 2022  9:29AM  241379        15            Mizner Bioscience, LLC   
8  Jul  8 2022  9:29AM  241382        10                 ISDIN Corporation   
9  Jul  8 2022  9:26AM  241380        12                         Hush Hush   

  Released Executing Completed  
0        1                      
1        6                      
2        1                      
3        1                      
4      159         9            
5       39                      
6        1         6            
7        9         2            
8        8                      
9       10

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America     
1  Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.   
2  Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.   
3  Jul  8 2022 10:05AM  241386        10                       Emerginnova   
4  Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.   

  Released Executing Completed  
0        1                      
1        6                      
2        1                      
3        1                      
4      159         9

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America     
1  Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.   
2  Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.   
3  Jul  8 2022 10:05AM  241386        10                       Emerginnova   
4  Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       6.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4     159.0        9.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jul  8 2022 10:22AM  241389        20  Sartorius Stedim North America     
1  Jul  8 2022 10:22AM  241387        10                  Eywa Pharma Inc.   
2  Jul  8 2022 10:18AM  241388        21                 NBTY Global, Inc.   
3  Jul  8 2022 10:05AM  241386        10                       Emerginnova   
4  Jul  8 2022  9:52AM  241378        15              Person & Covey, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       6.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4     159.0        9.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4344573     227.0        5.0        0.0
102         965249       4.0        0.0        0.0
12          241380      10.0        0.0        0.0
15          965517     183.0       25.0        0.0
16          241373       0.0        1.0        0.0
18          241351       0.0        1.0        0.0
19          241366      13.0       20.0        3.0
20          724084      20.0       15.0       25.0
21         1206771       3.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4344573     227.0        5.0        0.0
1       102   965249       4.0        0.0        0.0
2        12   241380      10.0        0.0        0.0
3        15   965517     183.0       25.0        0.0
4        16   241373       0.0        1.0        0.0
5        18   241351       0.0        1.0        0.0
6        19   241366      13.0       20.0        3.0
7        20   724084      20.0       15.0       25.0
8        21  1206771       3.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     227.0        5.0        0.0
1       102       4.0        0.0        0.0
2        12      10.0        0.0        0.0
3        15     183.0       25.0        0.0
4        16       0.0        1.0        0.0
5        18       0.0        1.0        0.0
6        19      13.0       20.0        3.0
7        20      20.0       15.0       25.0
8        21       3.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  227.0
1       102  Released    4.0
2        12  Released   10.0
3        15  Released  183.0
4        16  Released    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102    12     15   16   18    19    20   21
Status                                                       
Completed    0.0  0.0   0.0    0.0  0.0  0.0   3.0  25.0  0.0
Executing    5.0  0.0   0.0   25.0  1.0  1.0  20.0  15.0  2.0
Released   227.0  4.0  10.0  183.0  0.0  0.0  13.0  20.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102    12     15   16   18    19    20   21
0          Completed    0.0  0.0   0.0    0.0  0.0  0.0   3.0  25.0  0.0
1          Executing    5.0  0.0   0.0   25.0  1.0  1.0  20.0  15.0  2.0
2           Released  227.0  4.0  10.0  183.0  0.0  0.0  13.0  20.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102    12     15   16   18    19    20   21
0  Completed    0.0  0.0   0.0    0.0  0.0  0.0   3.0  25.0  0.0
1  Executing    5.0  0.0   0.0   25.0  1.0  1.0  20.0  15.0  2.0
2   Released  227.0  4.0  10.0  183.0  0.0  0.0  13.0  20.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()